# LangSmith 데이터셋 생성

자체 RAG 평가용 데이터셋을 구축하는 방법에 대해 알아보겠습니다.

먼저, 데이터셋 구축을 위해서는 크게 3가지 과정에 대한 이해가 필요합니다.

Case: Retrieval 이 Question 에 Relevant 한지 평가

> Question - Retrieval

![](./assets/eval-03.png)

Case: Answer 이 Question 에 Relevant 한지 평가

> Question - Answer   

![](./assets/eval-04.png)

Case: Answer 가 Retrieval 된 문서 안에서 답변하였는지 (Hallucination Check)

> Retrieval - Answer

![](./assets/eval-05.png)

따라서, `Question`, `Retrieval`, `Answer` 의 3가지 정보가 필요한 것이 일반적이지만, `Retrieval` 에 대한 Ground Truth 구축이 사실상 어렵습니다.

만약, `Retrieval` 에 대한 Ground Truth 가 존재한다면, 모두 데이터셋으로 저장하여 활용하고, 그렇지 않다면 `Question`, `Answer` 만으로 데이터셋을 구축하여 활용할 수 있습니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# # LangSmith 추적을 설정합니다. https://smith.langchain.com
# from langchain_teddynote import logging

# # 프로젝트 이름을 입력합니다.
# logging.langsmith("Low RAG Eval")

LangSmith 추적을 시작합니다.
[프로젝트명]
Low RAG Eval


## 데이터셋 생성

`inputs` 와 `outputs` 를 활용하여 데이터셋을 생성합니다.

데이터셋은 `question` 과 `answer` 로 구성됩니다.

### Ref
https://www.easylaw.go.kr/CSP/CnpClsMain.laf?csmSeq=629&ccfNo=1&cciNo=1&cnpClsNo=1 : 법률에 관한 질문을 쉽게 답변해줘서 참고함!

In [3]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "보증금 5천만원에 임차한 집에서 8개월째 살고 있습니다. 집주인이 주변시세가 많이 올랐다며, 보증금을 천만원을 더 올려달라고 합니다. 보증금을 올려주어야 하나요?",
    "임차하여 살고 있는 집이 경매로 넘어갔다고 합니다. 보증금을 돌려받을 수 있을까요?",
    "임차한 집에 거주하던 중 제 돈으로 집수리를 했습니다. 나중에 돌려받을 수 있나요?",
    "임대차계약을 하고 살고 있는 집이 압류되었는데, 집을 비워줘야 하나요?",
    "보일러가 고장이 났는데 집주인은 임차인인 제가 고치는 것이라고 하여 어쩔 수 없이 제 돈으로 보일러를 고쳤는데, 이런 경우에도 임차인이 고쳐야 하나요?",
    "계약 기간의 만료되어 이사 가려고 하는데 보증금은 새로운 세입자가 들어오면 그 때 준다고 합니다. 집주인의 말을 믿고 지금 이사를 가도 될까요?",
    "보증금 5천만원에 임차한 집에서 8개월째 살고 있습니다. 집주인이 주변시세가 많이 올랐다며, 보증금을 천만원을 더 올려달라고 합니다. 보증금을 올려주어야 하나요?",
    "주택임대차계약을 체결하고 주민등록 이전과 함께 이사를 하여 거주하고 있었습니다. 그런데 갑자기 어떤 사람이 ‘이 집은 명의신탁된 것으로 진짜 주인은 나다’라며 이사 갈 것을 요구하고 있습니다. 어떻게 해야 하나요?",
    "주택 임대차 계약을 체결하려는데, 주변에서 계약기간을 1년으로 하라고 합니다. 어떻게 하는 것이 좋을까요?",
    "임대차 계약 기간이 만료되어 집주인에게 보증금을 돌려달라고 하니, 새로운 세입자를 구할 때까지 기다려 달라고 합니다. 보증금을 돌려받을 때까지 그 집에서 계속 거주해도 월세는 안 내도 되나요?"
]

# 질문에 대한 답변 목록
outputs = [
    """올려주지 않아도 됩니다.
임대인의 보증금 증액청구는 임대차계약 또는 약정한 보증금의 증액이 있은 후 1년 이내에는 할 수 없습니다.
설사 1년이 지나서 증액을 청구한다 하더라도 약정한 보증금의 20분의 1의 금액을 초과하여 청구할 수 없습니다.
따라서 계약기간이 1년이 지나고 질문과 같이 보증금이 5천만원인 경우에는 250만원을 넘지 않는 범위에서는 증액을 요구할 수 있습니다.
◇ 임대인의 차임·보증금 증액
☞ 임대인은 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증가나 경제사정의 변동으로 인하여 적절하지 않게 된 경우에는 장래에 대하여 그 증액을 요구할 수 있습니다.
◇ 임대인이 요구할 수 있는 차임·보증금의 증액 한도
☞ 증액 한도는 약정한 차임·보증금의 20분의 1을 넘지 못합니다.
◇ 임대인이 증액 한도를 초과하여 증액을 요구하는 경우
☞ 계약기간이 1년이 지났고 보증금이 5천만원인데 집주인이 보증금 1천만원의 인상을 계속 요구하는 경우에는 법원에 가서 보증금의 5%인 250만원을 공탁하면 차임의 연체를 면하게 되고 그 집에서 계속 살 수 있습니다.""",
    """대항력과 확정일자를 받았다면 임차인은 우선변제권을 갖고, 이 경우 임차인은 임차주택이 경매 또는 공매에 붙여졌을 때 그 경락대금에서 다른 후순위권리자보다 우선하여 보증금을 변제받을 수 있습니다.
◇ 우선변제권
☞ 우선변제권이란 임차인이 보증금을 다른 채권자보다 우선 변제받을 수 있는 권리를 말합니다.
☞ 임차인은 ① 대항요건(주택의 인도 및 전입신고)과 ② 임대차계약서상의 확정일자를 갖춘 경우에는 우선변제권을 갖습니다.
◇ 확정일자
☞ 확정일자란 증서가 작성된 날짜에 주택임대차계약서가 존재하고 있음을 증명하기 위해 법률상 인정되는 일자를 말합니다.
☞ 임차인, 임차인의 대리인 등 주택임대차계약서의 소지인은 주택소재지의 읍사무소, 면사무소, 동 주민센터 또는 시·군·구의 출장소에서 확정일자를 부여받을 수 있습니다.
☞ 확정일자를 받기 위해서는 ① 임대인·임차인의 인적사항, 임대차 목적물, 임대차 기간, 보증금 등이 적혀 있고, ② 계약당사자의 서명 또는 기명날인이 되어 있는 완성된 문서여야 합니다.
☞ 주택임대차의 주택과 그 기간 등이 기재되어 있지 않은 영수증 등에 확정일자를 받더라도 우선변제권의 효력은 발생하지 않으므로 주의를 요합니다.
◇ 우선변제권의 발생시기
☞ 임차인이 주택의 인도와 전입신고를 마친 당일 또는 그 이전에 주택임대차계약서에 확정일자를 갖춘 경우에는 주택의 인도와 전입신고를 마친 다음날 오전 0시부터 우선변제권이 생깁니다.""",
    """임차인이 집수리를 위해 지출한 필요비나 유익비는 임대인에게 그 비용을 반환 청구할 수 있습니다.
임대차 계약이 종료된 후에도 필요비나 유익비를 반환받지 않았다면 주택의 인도를 거부할 수 있습니다.
◇ 필요비 상환청구
☞ “필요비”란 임대차계약이 목적에 따라 임차주택을 사용·수익하는 데 적당한 상태를 보존, 유지하기 위해 필요한 모든 비용을 말합니다.
☞ 여기에는 임대인의 동의 없이 지출한 비용도 포함됩니다. 임차인은 임차주택의 보존에 관해 필요비를 지출한 때에는 비용이 발생한 즉시 임대인에게 그 비용을 청구할 수 있습니다.
☞ 다만, 임차인이 별 비용을 들이지 않고 손쉽게 고칠 수 있을 정도의 사소한 것이어서 임차인의 사용·수익을 방해할 정도의 것이 아니라면 그 수선의무는 임차인이 부담합니다.
◇ 유익비 상환청구
☞ “유익비상환청구권”이란 임차인이 임대차관계로 임차주택을 사용·수익하던 중 그 객관적 가치를 증가시키기 위해 투입한 비용이 있는 경우 임대차 종료 시에 그 가액의 증가가 현존한 때에 한해 임대인에게 임대인의 선택에 따라 임차인이 지출한 금액이나 그 증가액의 상환을 청구할 수 있는 것을 말합니다.
☞ 따라서 유익비의 상환은 임차인이 임차기간 중에 지출한 유익비에 한하여 인정되고, 임차인이 유익비를 지출하여 증가된 가액이 임대차 종료 시에 현존해야 청구할 수 있습니다.""",
"""
압류만 되어 있는 상태라면 당장 집을 비워 줄 필요는 없습니다. 또한 대항력을 갖춘 경우라면 추후 경매가 되어도 비워주지 않아도 됩니다.
해당 임대차는 그 등기가 없더라도, 임차인이 임대차 계약 후 ① 주택을 인도받고 ② 주민등록을 마친 때에는 그 다음날부터 대항력이 생깁니다. 따라서 그 후의 권리자에게 대항할 수 있습니다.
대항력을 보유한 임차인은 제3자, 즉 임차 주택의 양수인, 임대할 권리를 승계한 사람, 그 밖에 임차 주택에 관해 이해관계를 가지고 있는 사람에게 임대차의 내용을 주장할 수 있습니다.
따라서 살고 있는 집이 압류되었고, 추후 경매가 된다 하더라도 대항력을 갖춘 경우에는 걱정하실 필요가 없습니다.
다만, 선순위권자(예를 들어 대항력이 발생하는 일자보다 앞서 등기가 되어 있는 담보권자가 있는 경우)가 있는 경우에는 대항력을 갖추었다 하더라도 경매 시 문제가 발생할 수 있으니 이를 확인해야 합니다.
◇ 대항력
☞ “대항력”이란 임차인이 제3자 즉 임차 주택의 양수인, 임대할 권리를 승계한 사람, 그 밖에 임차주택에 관해 이해관계를 가지고 있는 사람에 대하여 임대차의 내용을 주장할 수 있는 법률상의 힘을 말합니다.
☞ 임대차는 임대차 등기를 하지 않더라도, 임차인이 ① 주택의 인도(이사)와 ② 주민등록을 마친 때에는 그 다음날부터 대항력이 생깁니다.
◇ 주택의 인도
☞ “주택의 인도”란 점유의 이전을 말하는데, 주택에 대한 사실상의 지배가 임대인으로부터 임차인에게로 이전하는 것을 말합니다. 즉 임차인이 입주해서 살고 있는 것을 뜻합니다.
☞ 만약 실제로 임차인이 임차 주택으로 이사를 해서 거주하지 않고 있다면 「주택임대차보호법」의 보호를 받을 수 없게 됩니다.
""",
"""
집주인이 고쳐야 합니다. 임대인은 임대차계약이 존속하는 동안 임차인에 대하여 임대물의 사용·수익에 필요한 상태를 유지해야 할 의무가 있습니다. 보일러는 임대물의 사용·수익에 필수적인 사항이므로 집주인인 임대인은 이를 고쳐줘야 합니다.
◇ 임대인의 의무
☞ 임대물에 하자가 생겼을 경우에는 임대인은 이를 수리하여 완전한 상태로 임차인이 사용할 수 있도록 해야 합니다.
☞ 그러나 비용부담이 매우 적은 사소한 하자나 소모품 교환 정도는 임차인이 부담해야 합니다.
◇ 임대인이 수리를 거부하는 경우
☞ 임차인은 하자를 수리한 후 그 비용을 임대인에게 청구할 수 있습니다.
☞ 또한, 임차인은 계약을 해지하여 보증금의 반환을 임대인에게 청구할 수 있습니다. 그리고 임차인은 임차 주택의 하자 때문에 생긴 손해와 수리를 하거나 계약을 해지하면서 생기는 손해에 대한 배상도 임대인에게 청구할 수 있습니다.
""",
"""
「주택임대차보호법」은 주택의 인도와 주민등록을 대항력의 취득 및 존속 요건으로 하고 있기 때문에 임차인이 임대차가 종료되었음에도 보증금을 돌려받지 못하고 이사를 가게 되면 종전에 취득하였던 대항력 및 우선변제권이 상실되므로 보증금을 돌려받기 어려워지게 됩니다.
이러한 문제를 해결하기 위해 임차권등기명령제도는 법원의 집행명령에 따른 등기를 마치면 임차인에게 대항력 및 우선변제권을 유지하게 하면서 임차주택에서 자유롭게 이사할 수 있게 하는 제도입니다.
◇ 임차보증금의 반환 및 임차주택의 반환
☞ 임대차가 종료되더라도 임차인이 보증금을 돌려받을 때까지는 임대차관계가 존속하는 것으로 간주되므로, 임대인과 임차인은 임대차계약상의 권리의무를 그대로 가지게 됩니다.
☞ 임대차가 종료되었는데도 집주인이 보증금을 돌려주지 않는 경우에는 이사를 가면 대항력과 우선변제권이 없어지기 때문에 보증금을 반환받을 때까지 이사를 가지 않는 것이 좋습니다.
◇ 임차권등기명령제도
☞ 임차인은 임대차가 종료된 후 보증금이 반환되지 않은 경우에 임차권 등기명령을 신청할 수 있는 권한을 얻게 됩니다.
임차권등기명령을 받아서 임차권등기를 해 놓으면 대항력과 우선변제권을 취득하거나 유지할 수 있게 됩니다.
따라서 임차인은 임차권등기를 마친 후 임차주택을 인도하고 이사를 가더라도 대항력과 우선변제권을 유지할 수 있게 됩니다.
또한, 차임지급의무를 면하고, 임대인이 보증금반환채권을 지체한 것에 대한 지연손해금채권을 가지게 됩니다.
☞ 임차인은 임차권등기명령의 신청과 그에 따른 임차권등기와 관련하여 든 비용을 임대인에게 청구할 수 있습니다.
☞ 임차권등기명령제도는 가까운 법원의 민원실에서 안내받을 수 있습니다.
""",
"""
올려주지 않아도 됩니다.
임대인의 보증금 증액청구는 임대차계약 또는 약정한 보증금의 증액이 있은 후 1년 이내에는 할 수 없습니다.
설사 1년이 지나서 증액을 청구한다 하더라도 약정한 보증금의 20분의 1의 금액을 초과하여 청구할 수 없습니다.
따라서 계약기간이 1년이 지나고 질문과 같이 보증금이 5천만원인 경우에는 250만원을 넘지 않는 범위에서는 증액을 요구할 수 있습니다.
◇ 임대인의 차임·보증금 증액
☞ 임대인은 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증가나 경제사정의 변동으로 인하여 적절하지 않게 된 경우에는 장래에 대하여 그 증액을 요구할 수 있습니다.
◇ 임대인이 요구할 수 있는 차임·보증금의 증액 한도
☞ 증액 한도는 약정한 차임·보증금의 20분의 1을 넘지 못합니다.
◇ 임대인이 증액 한도를 초과하여 증액을 요구하는 경우
☞ 계약기간이 1년이 지났고 보증금이 5천만원인데 집주인이 보증금 1천만원의 인상을 계속 요구하는 경우에는 법원에 가서 보증금의 5%인 250만원을 공탁하면 차임의 연체를 면하게 되고 그 집에서 계속 살 수 있습니다.
""",
"""
건물을 인도(이사)받은 상태이고, 그 주소로 주민등록을 완료한 경우에는 크게 걱정할 필요가 없습니다.
해당 주택의 등기부상의 소유자인 명의수탁자는 대외적으로 적법한 소유자로 인정되고, 그가 행한 주택에 대한 처분 및 관리행위는 유효하기 때문입니다.
설사 명의신탁자(진짜 주인)가 명의신탁을 해지하고 소유권이전등기를 마친 후 주택의 반환을 요구해도 임차인은 그 요구에 따를 필요가 없습니다.
왜냐하면, 이 경우 명의신탁자는 명의수탁자의 지위를 승계한 것으로 보기 때문입니다.
◇ 명의신탁
☞ 소유관계를 공시(公示)하도록 되어 있는 재산에 대하여 소유자 명의를 진짜 소유자가 아닌 다른 사람 이름으로 해놓는 것을 말합니다.
☞ 명의신탁이 된 재산의 소유관계는 신탁자(진짜 소유자)와 수탁자 사이에서는 소유권이 그대로 신탁자에게 있지만, 대외관계 또는 제3자에 대한 관계에서는 소유권이 수탁자(가짜 소유자)에게 있습니다.
◇ 명의수탁자와의 임대차 계약 체결
☞ 주택의 명의수탁자와 임대차계약을 체결하는 경우에는 명의수탁자가 등기부상의 소유자와 동일한지만 확인하면 됩니다.
☞ 명의수탁자는 명의신탁의 법리에 따라 대외적으로 적법한 소유자로 인정되고, 그가 행한 신탁 목적물에 대한 처분 및 관리행위는 유효하기 때문입니다.
""",
"""
계약기간을 1년으로 하시더라도 최소한 2년의 계약기간을 보장받을 수 있습니다. 그리고 임차인은 계약대로 1년의 기간을 주장할 수도 있습니다.
◇ 임대차계약 기간
☞ 주택 임대차 계약을 체결하는 경우 계약서에 계약 기간을 정하지 않았거나 기간을 2년 미만으로 정했더라도 임차인은 최소한 2년의 기간을 보장받을 수 있습니다.
☞ 다만, 임차인은 2년 미만으로 정한 기간이 유효함을 주장할 수 있으므로 만약 계약기간을 1년으로 정했다면, 임차인은 2년 동안 임대차 기간을 보장받으면서도 필요한 경우에는 1년 후에 계약을 해지하고 임대인에게 보증금을 돌려받을 수 있습니다.
☞ 그러나 1년의 계약을 하더라도 임대인은 이를 계약의 내용으로 주장할 수 없습니다. 「주택임대차보호법」제4조는 편면적 강행규정으로 임차인에게 불리한 것으로 효력이 없기 때문입니다
""",
"""
그 집에서 계속 사는 동안은 월세를 계속 내야 합니다. 임대차 계약이 종료된 후에 임차인이 주택을 넘겨주는 의무와 임대인의 보증금 반환 의무는 동시에 행해져야 합니다.
◇ 임차주택의 반환 및 임차보증금의 반환
☞ 임대차기간이 끝난 경우에도 임차인이 보증금을 반환받을 때까지는 임대차관계가 존속되는 것으로 봅니다. 따라서 임대인과 임차인은 임대차계약상의 권리의무를 그대로 가지게되므로 집에서 계속 거주하는 동안은 월세를 내야 합니다.
◇ 임차권등기명령
☞ 임차인은 임대차가 종료된 후 보증금을 반환받지 못한 경우 임차권등기명령을 신청할 수 있는 권한을 얻게 되고, 임대차등기명령에 따라 임차권등기를 마치면 대항력과 우선변제권을 취득하거나 유지할 수 있게 됩니다.
"""
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()

,question,answer
0,보증금 5천만원에 임차한 집에서 8개월째 살고 있습니다. 집주인이 주변시세가 많이 ...,올려주지 않아도 됩니다.\n임대인의 보증금 증액청구는 임대차계약 또는 약정한 보증금...
1,임차하여 살고 있는 집이 경매로 넘어갔다고 합니다. 보증금을 돌려받을 수 있을까요?,"대항력과 확정일자를 받았다면 임차인은 우선변제권을 갖고, 이 경우 임차인은 임차주택..."
2,임차한 집에 거주하던 중 제 돈으로 집수리를 했습니다. 나중에 돌려받을 수 있나요?,임차인이 집수리를 위해 지출한 필요비나 유익비는 임대인에게 그 비용을 반환 청구할 ...
3,"임대차계약을 하고 살고 있는 집이 압류되었는데, 집을 비워줘야 하나요?",\n압류만 되어 있는 상태라면 당장 집을 비워 줄 필요는 없습니다. 또한 대항력을 ...
4,보일러가 고장이 났는데 집주인은 임차인인 제가 고치는 것이라고 하여 어쩔 수 없이 ...,\n집주인이 고쳐야 합니다. 임대인은 임대차계약이 존속하는 동안 임차인에 대하여 임...


In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

혹은 이전의 튜토리얼에서 생성한 Synthetic Dataset 을 활용할 수 있습니다.

아래 코드는 업로드한 HuggingFace Dataset 을 활용하는 예시입니다.

(참고) 아래의 주석을 풀고 실행하여 `datasets` 라이브러리를 업데이트 후 진행해 주세요. 

In [ ]:
# !pip install -qU datasets

In [4]:
# import pandas as pd
# from datasets import load_dataset, Dataset
# import os

# # huggingface Dataset에서 repo_id로 데이터셋 다운로드
# dataset = load_dataset(
#     "junn991/Low-RAG-Eval-dataset",  # 데이터셋 이름
#     token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # private 데이터인 경우 필요합니다.
# )

# # 데이터셋에서 split 기준으로 조회
# huggingface_df = dataset["korean_v1"].to_pandas()
# huggingface_df.head()

KeyError: 'HUGGINGFACEHUB_API_TOKEN'

## LangSmith 테스트를 위한 Dataset 생성

- `Datasets & Testing` 에 새로운 데이터셋을 생성합니다.

![](./assets/eval-06.png)

csv 파일에서 LangSmith UI를 사용하여 직접 데이터셋을 생성할 수도 있습니다.

자세한 사항은 아래 문서를 참고해 주세요.

- [LangSmith UI 문서](https://docs.smith.langchain.com/evaluation/faq/manage-datasets)

In [4]:
from langsmith import Client

client = Client()
dataset_name = "Low_RAG_Eval_Dataset"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset


# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

데이터셋에 예제를 나중에 추가할 수 있습니다.
- 정답이 아닌것도 추가함

In [6]:
# # 새로운 질문 목록
# new_questions = [
#     "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
#     "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
# ]

# # 새로운 답변 목록
# new_answers = [
#     "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
#     "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
# ]

# # UI에서 업데이트된 버전 확인
# client.create_examples(
#     inputs=[{"question": q} for q in new_questions],
#     outputs=[{"answer": a} for a in new_answers],
#     dataset_id=dataset.id,
# )

축하합니다! 이제 데이터셋이 준비되었습니다.
- 정답의 기준이 되는 데이터셋..!